In [1]:
import spacy
import re
nlp = spacy.load("/usr/local/lib/python3.6/dist-packages/en_core_web_sm/en_core_web_sm-2.0.0")

In [2]:
doc = "I like the opera. On sundays I go there."
doc_nlp = nlp(doc)

In [15]:
for s in doc_nlp.sents:
    nps = s.noun_chunks
    for np in nps:
        print(np.start)
        print(np.end)
        print(s[np.start: np.end])
        cnp = re.sub(' ', '_', np.lemma_)
        print(cnp)

0
1
I
-PRON-
2
4
the opera
the_opera
6
7

sunday
7
8

-PRON-


Test preparing sentence for elmo embeddings.

In [1]:
sent = 'The arm_chair_radio of the couch is in the greek_mythology .'.split()
sent_terms = [(1, ['arm', 'chair', 'radio'], 1), (1, ['couch'], 4), (1, ['greek', 'mythology'], 8)]
sent1 = 'Sometimes machine_learning is difficult'.split()
sent_terms1 = [(1, ['machine', 'learning'], 1)]

In [2]:
from typing import *
from collections import defaultdict
def prepare_sentence(sent: List[str],
                     sent_terms: List[Tuple[str, List[str], int]]
                     ) -> Tuple[List[str], List[List[int]]]:
    """Make sent where the words of terms are split into single words.

    Args:
        sent: The original sentence.
        sent_terms: A list of terms in the sentence. Each term in the
        sentence is represented by a Tuple in the form
        (term-id, list of words in term, index of term in the sentence)
    Return:
        1. The sentence, but all multiword terms are split into single
        words again.
        2. Dict of term-indices of the form: {i: [indices_in_sent]}
        i is the position of the term in sent_terms. 
    """
    for i in range(len(sent_terms))[::-1]:
        term = sent_terms[i]
        term_idx = term[2]
        term_words = [t+str(i) for t in term[1]]
        sent[term_idx] = term_words
    
    # Flatten list.
    flat_sent = []
    for w in sent:
        if isinstance(w, list):
            for v in w:
                flat_sent.append(v)
        else:
            flat_sent.append(w)
    
    # Get indices and clean sent. Only works if there are less then 10 terms in one sentence.
    prepped_sent = []
    indices_in_sent = defaultdict(list)
    for i in range(len(flat_sent)):
        w = flat_sent[i]
        if w[-1].isdigit():
            term_idx = int(w[-1])
            indices_in_sent[term_idx].append(i)
            w = w[:-1]
            prepped_sent.append(w)
        else:
            prepped_sent.append(w)
    
    return prepped_sent, indices_in_sent

In [3]:
prepare_sentence(sent, sent_terms)

(['The',
  'arm',
  'chair',
  'radio',
  'of',
  'the',
  'couch',
  'is',
  'in',
  'the',
  'greek',
  'mythology',
  '.'],
 defaultdict(list, {0: [1, 2, 3], 1: [6], 2: [10, 11]}))

Test calculating the embedding of a term.

In [4]:
from numpy import mean
def get_term_emb(embs: List[List[float]],
                 term: List[int]
                 ) -> List[float]:
    """Get the embedding for the given term.

    For multiword terms, get the average embedding.
    
    Args:
        embs: A list of word embeddings.
        term: The indices of a term's words in the current sentence.
    Return:
        The embedding of the term.
    """
    print(term)
    term_embs = [embs[i] for i in term]
    print(term_embs)
    mean_emb = mean(term_embs, 0)
    return [float(f) for f in mean_emb] 

In [5]:
embs = [[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5], [0.6, 0.6, 0.6], [0.7, 0.7, 0.7], [0.8, 0.8, 0.8], [0.9, 0.9, 0.9], [0.1, 0.1, 0.1], [0.11, 0.11, 0.11], [0.12, 0.12, 0.12]]
term = [9, 10]

In [6]:
get_term_emb(embs, term)

[9, 10]
[[0.1, 0.1, 0.1], [0.11, 0.11, 0.11]]


[0.10500000000000001, 0.10500000000000001, 0.10500000000000001]

In [7]:
mean([[1, 2], [2, 3]], 0)

array([1.5, 2.5])

Test the merge dictionaries function.

In [12]:
import json
def merge_dicts(fpaths: List[str], path_out: str) -> None:
    """Merge n dictionaries to one dictionary and write to file.
    
    Args:
        fpaths: A list of paths to json files that contain the dictionaries.
        path_out: Path to the output json file.
    Return:
        Write a dictionary with the structure 
        Dict[int, Dict[int, List[float]]] to file. 
    """
    out_dict = {}
    for fp in fpaths:
        with open(fp, 'r', encoding='utf8') as f:
            cur_dict = json.load(f)
            for term_id in cur_dict:
                if term_id in out_dict:
                    cur_term_dict = cur_dict[term_id]
                    out_dict_term = out_dict[term_id]
                    for doc_id in embs_term:
                        if doc_id in out_dict_term[doc_id]:
                            print('Two files have the same doc_ids...there is something wrong!')
                        else:
                            out_dict_term[doc_id] = cur_term_dict[doc_id]
                else:
                    out_dict[term_id] = cur_dict[term_id]
    with open(path_out, 'w', encoding='utf8') as f:
        json.dump(out_dict)
    return outdict
                        

In [ ]:
import 